# Example: head-and-neck treatment plan optimization

![Coronal view on the head-and-neck CT/dose distribution](_static/coronal_view.png)

## Intro

Welcome to the pyanno4rt example notebook! <br><br> In this notebook, we will showcase the core functionality of our package using data from a head-and-neck example patient (available from the Github repository as .mat-files). The first part will present the simplest version of the code-based interface, followed by the UI-based interface in the second part.

## Import of the relevant classes

First, we import the base classes. Our package is designed for clarity and ease of use, wherefore it has only one class for initializing a treatment plan and one class for initializing the graphical user interface. Hence, the import statements are:

In [ ]:
from pyanno4rt.base import TreatmentPlan
from pyanno4rt.gui import GraphicalUserInterface

## Code-based interface

If you prefer to work with the command line interface (CLI) or an interactive development environment (IDE), you can initialize the <i>TreatmentPlan</i> class by hand. The parameter space of this class is divided into three parameter groups:
<ul>
    <li> <b>configuration parameters:</b> design parameters w.r.t general or external data settings for the treatment plan</li>
    <li> <b>optimization parameters:</b> design parameters w.r.t the components (objectives and constraints), method and solver for treatment plan optimization</li>
    <li> <b>evaluation parameters:</b> design parameters w.r.t the evaluation methods used for treatment plan assessment</li>
</ul>

Well then, let's create an instance of the <i>TreatmentPlan</i> class!

### Treatment plan initialization

For the sake of readability, we will define the parameter groups one by one (of course, you could also directly specify them in the base class arguments). Our package utilizes Python dictionaries for this purpose, which allow an efficient mapping between parameter names and values per group and promote a transparent setup and passing.

#### Setting up the configuration dictionary

We decide to label our plan 'Head-and-neck-example' and set the minimum logging level to 'info', which means that any debugging or warning messages will be suppressed. For the modality and the number of fractions, we stick to the default values 'photon' and 30. Since we have some MATLAB files available for the head-and-neck example patient, we provide the corresponding paths to the imaging and dose-influence matrix files (you may need to change them). Post-processing interpolation of the imaging data is not required, so we leave the parameter at None. Finally, we know that the dose-influence matrix has been calculated with a resolution of 5 mm in each dimension, so we set the dose resolution parameter accordingly.

In [ ]:
configuration = {
    'label': 'Head-and-neck-example', # Unique identifier for the treatment plan
    'min_log_level': 'info', # Minimum logging level
    'modality': 'photon', # Treatment modality
    'number_of_fractions': 30, # Number of fractions
    'imaging_path': './example_patient_data.mat', # Path to the CT and segmentation data
    'target_imaging_resolution': None, # Imaging resolution for post-processing interpolation of the CT and segmentation data
    'dose_matrix_path': './example_photon_dij.mat', # Path to the dose-influence matrix
    'dose_resolution': [5, 5, 5] # Size of the dose grid in [mm] per dimension
    }

Great, we have completely defined the first parameter group 👍

#### Setting up the optimization dictionary

Next, we need to describe how the treatment plan should be optimized (probably the most exciting, but also the most difficult part). In general, the final plan should apply a reasonably high dose to the planning target volumes (PTVs) while limiting the dose exposure to relevant organs at risk to prevent post-treatment complications. <br><br> To achieve this, we define objective functions for the left and right parotid glands ('PAROTID_LT' and 'PAROTID_RT'), for both PTVs ('PTV63' and 'PTV70'), and for the skin ('SKIN'), where 'Squared Overdosing' refers to a function that penalizes dose values above a maximum, and 'Squared Deviation' refers to a function that penalizes upward and downward deviations from a target. The definition of these functions is again based on a dictionary, the components dictionary: it takes the segment names from the imaging data as keys and sub-dictionaries as values, in which the component type ('objective' or 'constraint') and the component instance (or a list of instances) with the component's class name and parameters are set. <br><br> Once the components have been defined, we find ourselves in a trade-off situation, where a higher degree of fulfillment for one objective is usually accompanied with a lower degree of fulfillment for another. We can handle this by choosing the 'weighted-sum' method, which bypasses the multi-objective problem by multiplying each objective value with a weight parameter and then summing them up, effectively merging them into a scalar "total" objective function. This works well with the default solution algorithm, the 'L-BFGS-B' algorithm from the 'scipy' solver, so we pick that one. For the initialization of the fluence vector (holding the decision variables), we opt for 'target-coverage' to start off with a satisfactory dose level for the PTVs (alternatively we could have passed 'warm-start' and replaced None for the initial fluence vector with an array). We place a lower bound of 0 and no upper bound (None) on the fluence, matching its physical properties. As the final step, we limit the number of iterations to 500 and the tolerance (precision goal) for the objective function value to 0.001.

In [ ]:
optimization = {
    'components': { # Optimization components for each segment of interest
        'PAROTID_LT': {
            'type': 'objective',
            'instance': {
                'class': 'Squared Overdosing',
                'parameters': {
                    'maximum_dose': 25,
                    'weight': 100
                }
            }
        },
        'PAROTID_RT': {
            'type': 'objective',
            'instance': {
                'class': 'Squared Overdosing',
                'parameters': {
                    'maximum_dose': 25,
                    'weight': 100
                }
            }
        },
        'PTV63': { 
            'type': 'objective',
            'instance': {
                'class': 'Squared Deviation',
                'parameters': {
                    'target_dose': 63,
                    'weight': 1000
                }
            }
        },
        'PTV70': {
            'type': 'objective',
            'instance': {
                'class': 'Squared Deviation',
                'parameters': {
                    'target_dose': 70,
                    'weight': 1000
                }
            }
        },
        'SKIN': {
            'type': 'objective',
            'instance': {
                'class': 'Squared Overdosing',
                'parameters': {
                    'maximum_dose': 30,
                    'weight': 800
                }
            }
        }
    },
    'method': 'weighted-sum', # Single- or multi-criteria optimization method
    'solver': 'scipy', # Python package to be used for solving the optimization problem
    'algorithm': 'L-BFGS-B', # Solution algorithm from the chosen solver
    'initial_strategy': 'target-coverage', # Initialization strategy for the fluence vector
    'initial_fluence_vector': None, # User-defined initial fluence vector (only for 'warm-start')
    'lower_variable_bounds': 0, # Lower bounds on the decision variables
    'upper_variable_bounds': None, # Upper bounds on the decision variables
    'max_iter': 500, # Maximum number of iterations for the solvers to converge
    'tolerance': 0.001 # Maximum CPU time for the solvers to converge
    }

Yeah, this was a tough piece of work! If you have managed to complete the optimization dictionary, feel free to reward yourself with a cup of tea or coffee, maybe a small snack, and a relaxing short break before moving on ☕

#### Setting up the evaluation dictionary

It is not actually necessary to set up the evaluation dictionary if you are happy with the default values. However, we will initialize it for reasons of completeness. First, we select the DVH type 'cumulative' and request its evaluation at 1000 (evenly-spaced) points. With the parameters 'reference_volume' and 'reference_dose', we let the package calculate dose and volume quantiles at certain levels. By inserting an empty list for 'reference_dose', the levels are automatically determined. The last two parameters, 'display_segments' and 'display_metrics', can be used to filter the names of the segments and metrics to be displayed later in the treatment plan visualization. We also specify empty lists here to not exclude any segment or metric.

In [ ]:
evaluation = {
    'dvh_type': 'cumulative', # Type of DVH to be calculated
    'number_of_points': 1000, # Number of (evenly-spaced) points for which to evaluate the DVH
    'reference_volume': [2, 5, 50, 95, 98], # Reference volumes for which to calculate the inverse DVH values
    'reference_dose': [], # Reference dose values for which to calculate the DVH values
    'display_segments': [], # Names of the segmented structures to be displayed
    'display_metrics': [] # Names of the plan evaluation metrics to be displayed
    }

Congratulations, you have successfully set up all parameter dictionaries 🎉

#### Initializing the base class

Now let's finally put everything together into a complete <i>TreatmentPlan</i> instance.

In [ ]:
tp = TreatmentPlan(configuration, optimization, evaluation)

### Treatment plan workflow

In this section, we describe the standard workflow in which the generated treatment plan instance comes into play. Our package equips the instance with one method for each work step, which can be called parameter-free.

#### Configuring the plan

First, a successfully initialized treatment plan needs to be configured. By calling the <i>configure</i> method, the information from the configuration dictionary is transferred to internal instances of the configuration classes, which perform functional (logging, data management) and I/O tasks (processing of imaging data, preparation of data dictionaries). Note that a plan must be configured before it can be optimized.

In [ ]:
tp.configure()

#### Optimizing the plan

Afterwards, the treatment plan is ready for optimization. We call the <i>optimize</i> method, which generates the internal optimization classes, passes the optimization parameters from the dictionary, and at the end triggers the solver run. If machine learning model-based components are used, the model fitting would also take place here. In our example, no such components exist, which means that the optimization process starts immediately. Note that a plan must be optimized before it can be evaluated.

In [ ]:
tp.optimize()

#### Evaluating the plan

The penultimate step is the evaluation of the treatment plan, and following the previous logic, we have added an <i>evaluate</i> method for this purpose. Internally, this creates objects from the DVH and dosimetrics class, which take the parameters of the evaluation dictionary and activate the respective evaluation processes. Note that a plan must be evaluated before it can be visualized.

In [ ]:
tp.evaluate()

#### Visualizing the plan

We are now at the end of the standard workflow, and of course we would like to conclude by analyzing the results of the treatment plan optimization and evaluation both qualitatively and quantitatively. Our package features a visual analysis tool that provides three sets of visualizations: optimization problem analysis, data-driven model review, and treatment plan evaluation. By clicking on the activated buttons, you can open the plot windows. The visual analysis tool can easily be launched with the <i>visualize</i> method.

In [ ]:
tp.visualize()

Ideally, you should now see the window below.

![pyanno4rt visualizer](_static/visualizer.png)

(By the way, the top image in this notebook has been extracted from the CT/Dose slice plot 😉)

#### Shortcut: composing the plan

Many times you will just run all four of the above methods in sequence. To make this a little more convenient, the treatment plan can also be "composed" in a single step, using the appropriately named <i>compose</i> method (yes, we love music ❤️).

In [ ]:
tp.compose()

#### Updating parameter values

One last class functionality is the updating of parameter values with the <i>update</i> method. This comes in handy because each of the <i>configure</i>, <i>optimize</i> and <i>evaluate</i> methods is based on the corresponding parameter dictionary, so that, for example, the <i>evaluate</i> method can be called again after updating an evaluation parameter without repeating all the initialization and prior workflow steps. <br><br>
The <i>update</i> method takes a dictionary with key-value pairs as input, where the former are from the parameter dictionaries, and the latter are the new parameter values. We do not want to change the plan at this point, so we will just overwrite the modality and the DVH type with the previous values for illustration purposes. 

In [ ]:
tp.update({
    'modality': 'photon',
    'dvh_type': 'cumulative'
    })

#### Saving and loading treatment plans

Treatment plans generated within our package can be saved as a snapshot folder and loaded from there as a copycat. You can import the corresponding functions from the tools subpackage.

In [ ]:
from pyanno4rt.tools import copycat, snapshot

A snapshot automatically includes a JSON file with the parameter dictionaries, a compiled log file, and (if machine learning model-based components are used) subfolders with model configuration files. Optionally, you can specify whether to add the imaging data, the dose-influence matrix, and the model training data (this allows sharing an instance of <i>TreatmentPlan</i> with all input data). The name of the snapshot folder is specified by the treatment plan label from the configuration dictionary. <br><br> Assuming the snapshot is to be saved in the current path, the line below would create the minimum-sized version of a snapshot folder.

In [ ]:
snapshot(instance=tp, path='./', include_patient_data=False, include_dose_matrix=False, include_model_data=False)

Conversely, a snapshot that has been saved can be loaded back into a Python variable by calling the <i>copycat</i> function with the base class (which is always <i>TreatmentPlan</i>) and the folder path.

In [ ]:
tp_copy = copycat(base_class=TreatmentPlan, path='./Head-and-neck-example/')

## UI-based interface

Our package can also be accessed from a graphical user interface (GUI) if you prefer this option. There are many good reasons for using the GUI:
<ul>
    <li><b>Support with the initialization:</b> the GUI window provides the parameter names in a structured form and at the same time already defines parts of the parameter space.</li>
    <li><b>Handling of large parameter spaces:</b> the parameter space of a treatment plan can very quickly become high-dimensional, especially with many components (and even more with machine learning model-based components), making the code-based generation of the dictionaries more complex than the UI-based generation.</li>
    <li><b>Faster generation of treatment plans and cross-instance comparison:</b> due to the first two key points, among others, the GUI allows faster treatment plan initialization and workflow execution, and multiple plans can be saved and switched quickly for comparison.</li>
</ul>

And, of course, a GUI also simply looks good 😎

### GUI initialization

So, how can the GUI be called? Instead of initializing the <i>TreatmentPlan</i> class, we create an object of the <i>GraphicalUserInterface</i> class.

In [ ]:
gui = GraphicalUserInterface()

### GUI opening

Then, you can open the GUI window directly using the <i>launch</i> method.

In [ ]:
gui.launch()

Below you can see the main window of the GUI that should now appear.

![pyanno4rt GUI](_static/gui.png)

Without going into detail, the most important widgets shall be described here:
<ul>
    <li><b>Menu bar (upper row):</b> load/save a treatment plan, drop an existing treatment plan instance, instance selector, settings/info windows, exit the GUI</li>
    <li><b>Composer (left column):</b> tabs for setting the configuration, optimization and evaluation parameters</li>
    <li><b>Workflow (middle column):</b> action buttons for the workflow steps, toolbox for accessing generated data (e.g. log files) </li>
    <li><b>Viewer (right column):</b> Axial CT/dose preview, interactive DVH</li>
</ul>

Alternatively, you can launch the GUI directly with an instance.

In [ ]:
gui.launch(tp)

### Fetching treatment plans from the GUI

The GUI has an internal dictionary in which objects of the <i>TreatmentPlan</i> class generated from the interface are stored. These can also be retrieved after closing the GUI using the <i>fetch</i> method.

In [ ]:
tps_gui = gui.fetch()

## Outro

We very much hope that this little example illustrates the basic usage of our package for treatment plan optimization. If you have any questions or suggestions, or in the (hopefully unlikely) event that something does not work, please take a look at the "Help and support" section and drop us a line. We would also be happy if you leave a positive comment or recommend our work to others. <br><br> Thank you for using pyanno4rt 😊